In [1]:
import os


In [2]:
%pwd  # this tell us which path we are currently working , so based on the below output path we are working under the research file
os.chdir("C:\\datascience End to End Projects\\TV sales prediciton\\TV-sales-prediction")  #  but i would like to work with main ProjectML_with_MLFlow file , so for getting i step back in path inorder to enter the main project file i used this command os.chdir("../")
%pwd

'C:\\datascience End to End Projects\\TV sales prediciton\\TV-sales-prediction'

In [24]:
# Now iam creating the entity class which consist of config.yaml folder model trainer code part variables, along with that iam adding some more varaibles like alpha,l1_ratio,target_column inside my entity class
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str  # this target column is present inside the Schema.yaml file which it tells us the quality of the Wine based on the value it returns

In [25]:
from TV_sales.constants import *
from TV_sales.utils.common import read_yaml, create_directories

In [34]:
# this template we use for every stage like data_ingestion,data_validation,data_transformation, model trainer .. etc


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

# this is part of code for the Model trainerConfig which helps us to return the configuration
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer   # here iam reading the schema, params 
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        print("Artifacts folder of Model_trainer created successfully")

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            target_column = schema.name # here from schema iam taking the name which i will return through target_column
            
        )

        return model_trainer_config # here iam returning all variables from the configuration

In [35]:
# these packages i need in order to create my Model Trainer components 
import pandas as pd
import os
from TV_sales import logger
from sklearn.linear_model import LinearRegression  # here iam importing ElasticNet from sklearn
import joblib # here iam saving the model because i want to save the data

In [38]:
# now here iam defining a class called model trainer inside it will take ModelTrainerConfig
class ModelTrainer:
    print("initiated model training")
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    # here iam creating a methode which it will traine the model by using train and test dataset
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path) # here it is taking the paths of train and test dataset
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)  # here iam dropping my target column in train_x
        test_x = test_data.drop([self.config.target_column], axis=1)  # here iam dropping my target column in test_X
        train_y = train_data[[self.config.target_column]]  # here iam keeping the target column in train_y
        test_y = test_data[[self.config.target_column]]  # here iam keeping the target column in test_y


        lr = LinearRegression() # here i have created my Elastic model which it takes the alpha,l1_ratio, random state values 
        lr.fit(train_x, train_y) # here i have initiated the model training

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name)) # here are training my model iam just saving inside the folder Model_trainer which it will get create inside the artifacts

initiated model training


In [39]:
try:
    config = ConfigurationManager() # here iam initlizing my ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config() # here iam getting my get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config) # here iam  passing my  model_trainer_config to the ModelTrainer function
    model_trainer_config.train() # here iam training the model
except Exception as e:
    raise e

[2024-02-03 17:36:03,417: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-03 17:36:03,419: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-03 17:36:03,422: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-03 17:36:03,425: INFO: common: created directory at: artifacts]
[2024-02-03 17:36:03,427: INFO: common: created directory at: artifacts/model_trainer]
Artifacts folder of Model_trainer created successfully
